This notebook is for timing TRT model inference vs non-TRT model inference

In [1]:
%%bash

echo 'Convert TRT:'
CONVERT_TRT_DIR=../../../tmp/trt_end_to_end/convert/trt
find ${CONVERT_TRT_DIR} -type f | sort

## Setup

In [2]:
import sys
sys.path.append('../../..')

## Parameters

In [3]:
_NAME = 'retinanet'
_EPOCH = 16
_PRECISION_MODE = 'INT8'

In [4]:
from src.utils.trt_end_to_end_constants import *
_NAME, _EPOCH, _TIME = get_params(_NAME, _EPOCH)

Metadata

In [5]:
from src.utils.trt_end_to_end_constants import MD_FILE_FORMAT

md_filename = MD_FILE_FORMAT % (_NAME, _EPOCH, _TIME)
_train_dir = get_train_dir(_NAME, _EPOCH)
md_filepath = os.path.join(_train_dir, md_filename)

from src.meta.metadata import Metadata
ret, metadata = Metadata.from_md(md_filepath)
assert ret == 0

from pprint import pprint
pprint(vars(metadata))

## Data

In [6]:
from src.data.cifar100 import CLASSES, INPUT_SHAPE, load_data
import numpy as np

(train_images, train_labels), (test_images, test_labels) = load_data()
input_img = np.array([train_images[0]])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [7]:
input_img.shape

(1, 32, 32, 3)

## Helper functions

In [8]:
import tensorflow as tf
from tensorflow.python.platform import gfile
import time

# function to read a ".pb" model 
# (can be used to read frozen model or TensorRT model)
def read_pb_graph(model):
    with gfile.FastGFile(model,'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    return graph_def

## Test Params

In [9]:
_WARM_UP_TRIALS = 10000
_TRIALS = 10000

## TensorRT

In [10]:
trt_graph_dir = get_trt_graph_dir(_NAME, _EPOCH)
trt_graph_filename = TRT_GRAPH_FILE_FORMAT % (_NAME, _EPOCH, _TIME, _PRECISION_MODE)
trt_graph_filepath = os.path.join(trt_graph_dir, trt_graph_filename)
print(trt_graph_filepath)

../../../tmp/trt_end_to_end/convert/trt/retinanet/016/retinanet_epoch016_2019-09-03T19:31_trt_INT8.pb


In [11]:
# variable

# required import or it will fail with weird message
import tensorflow.contrib.tensorrt as trt

graph = tf.Graph()
with graph.as_default():
    with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.50))) as sess:
        # read TensorRT model
        trt_graph = read_pb_graph(trt_graph_filepath)

        # obtain the corresponding input-output tensor
        tf.import_graph_def(trt_graph, name='')
        input = sess.graph.get_tensor_by_name(metadata.input_names[0])
        output = sess.graph.get_tensor_by_name(metadata.output_names[0])
        
        # warmup
        total_time = 0
        start = time.time()
        n = _WARM_UP_TRIALS
        print("Warming up for %d trials..." % n)
        last = None
        longest_line = 0
        for i in range(n):
            t1 = time.time()
            # CALL
            out_pred = sess.run(output, feed_dict={input: input_img})
            delta_time = time.time() - t1
            total_time += delta_time
            running_avg = (time.time() - start) / (i + 1)
            
            pctg = int(100 * i / n)
            if last is None or pctg > last:
                log = '['
                log += '=' * pctg
                log += '>'
                log += '.' * (100 - pctg - 1)
                log += ']'
                remaining_time = running_avg * (n - i - 1)
                log += (' ETA: %.3fs' % remaining_time)
                longest_line = max(len(log), longest_line)
                if len(log) < longest_line:
                    log += ' ' * (longest_line - len(log))
                print('\r', log, sep='', end='\r')
                last = pctg
        print()
        print()

        # test
        start = time.time()
        total_time = 0
        n = _TRIALS
        print("Testing for %d trials..." % n)
        last = None
        for i in range(n):
            t1 = time.time()
            # CALL
            out_pred = sess.run(output, feed_dict={input: input_img})
            delta_time = time.time() - t1
            total_time += delta_time
            running_avg = total_time / (i + 1)
            
            pctg = int(100 * i / n)
            if last is None or pctg > last:
                log = '['
                log += '=' * pctg
                log += '>'
                log += '.' * (100 - pctg - 1)
                log += ']'
                remaining_time = running_avg * (n - i - 1)
                log += (' ETA: %.3fs' % remaining_time)
                longest_line = max(len(log), longest_line)
                if len(log) < longest_line:
                    log += ' ' * (longest_line - len(log))
                print('\r', log, sep='', end='\r')
                last = pctg
        print()
            
        avg_time_tensorRT = total_time / n
        print("TRT %s avg time: %s" % (_PRECISION_MODE, avg_time_tensorRT))

W0904 01:48:30.099076 139810911356736 deprecation.py:323] From <ipython-input-8-3a99888420c0>:8: FastGFile.__init__ (from tensorflow.python.platform.gfile) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.gfile.GFile.


Warming up for 10000 trials...
[===================================================================================================>] ETA: 1.285s    

Testing for 10000 trials...
[===================================================================================================>] ETA: 1.251s    
TRT INT8 avg time: 0.012632353043556213
